In [33]:
import pandas as pd
import geopandas as gpd
import pull_noaa
import read_file
import load_weather
import load_interventions

In [34]:
noaa = pull_noaa.main()
noaa = noaa[['date', 'PRCP', 'TMAX', 'TMIN', 'lat', 'lon']]

reading...
fixing date...


In [35]:
shape = gpd.read_file('../data_raw/tl_2017_us_county/tl_2017_us_county.shp')
shape = shape[['NAMELSAD', 'geometry']]

In [36]:
shape.NAMELSAD = shape.NAMELSAD.str.upper()
noaa.lat = pd.to_numeric(noaa.lat)
noaa.lon = pd.to_numeric(noaa.lon)

In [37]:
noaa = gpd.GeoDataFrame(noaa, crs={'init': 'epsg:4326'}, geometry=gpd.points_from_xy(x=noaa.lon, y=noaa.lat))

In [ ]:
weather = gpd.sjoin(noaa, shape, how='inner', op='intersects')

In [ ]:
weather = weather.fillna(weather.mean(), inplace=True)